In [46]:
import pandas as pd
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from time import time
import io
import re

#import pickle
#from csv import reader
#import matplotlib.pyplot as plt

#from matplotlib import colors
#from matplotlib.ticker import PercentFormatter

from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten

from datetime import datetime

from collections import defaultdict

#!pip install sentencepiece
import sentencepiece

In [47]:
#!pip install transformers

## T5 base model

### Helpful links:
- https://huggingface.co/docs/transformers/model_doc/t5
- https://github.com/datasci-w266/2021-fall-main/blob/master/materials/Bert/BERT_T5_NER_2_3_030521.ipynb

In [50]:
from transformers import BertTokenizer, TFBertModel, T5Tokenizer, TFT5ForConditionalGeneration


In [51]:
t5_model = 't5-base'

t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)
t5 = TFT5ForConditionalGeneration.from_pretrained(t5_model)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [52]:
transcripts = pd.read_csv('cleaned_transcripts.csv').drop(['Unnamed: 0'], axis=1)
transcripts

,title,link,description,duration,podcast_id,transcript,cleaned_transcripts
0,Changes To Texas Voting Rules Worry Older Vote...,https://www.npr.org/2022/03/02/1084043800/chan...,"Election officials said more than 15,000 mail-...",842.0,1084043800,"[<div aria-label=""Transcript"" class=""transcrip...","Hi. My name is Joey (ph), and I'm a bike mech..."
1,"At The State Of The Union, Biden Wooed Moderat...",https://www.npr.org/2022/03/02/1083868631/at-t...,"In his speech, the president emphasized polici...",1141.0,1083868631,"[<div aria-label=""Transcript"" class=""transcrip...",Hey there. It's the NPR POLITICS PODCAST. I'...
2,"""Awake Not Woke"": How Republicans Are Defining...",https://www.npr.org/2022/02/28/1083491401/-awa...,At last week's Conservative Political Action C...,828.0,1083491401,"[<div aria-label=""Transcript"" class=""transcrip...","(Speaking Catalan). And I'm her mother, Gina ..."
3,Ketanji Brown Jackson Is The First Black Woman...,https://www.npr.org/2022/02/25/1083180663/keta...,Jackson's experiences working as a public defe...,1571.0,1083180663,"[<div aria-label=""Transcript"" class=""transcrip...","Hi, this is Teresa (ph) on Jekyll Island, Ga...."
4,US Responds To Russian Invasion Of Ukraine Wit...,https://www.npr.org/2022/02/24/1082922707/us-r...,In a speech at the White House Thursday aftern...,796.0,1082922707,"[<div aria-label=""Transcript"" class=""transcrip...",Hey there. It's the NPR POLITICS PODCAST. It...
...,...,...,...,...,...,...,...
2052,American Shadows,https://www.npr.org/2019/02/13/694463513/ameri...,Conspiracy theories are a feature of today's n...,2495.0,694463513,"[<div aria-label=""Transcript"" class=""transcrip...",If you've ever watched a horror movie in whic...
2053,High Crimes And Misdemeanors,https://www.npr.org/2019/02/25/697896407/high-...,"When Andrew Johnson became president in 1865, ...",1022.0,697896407,"[<div aria-label=""Transcript"" class=""transcrip...","Hey, I'm Ramtin Arablouei. I'm Rund Abdelfa..."
2054,The Forgotten War,https://www.npr.org/2019/02/12/694152504/the-f...,President Trump and North Korean Leader Kim Jo...,2053.0,694152504,"[<div aria-label=""Transcript"" class=""transcrip...",We're going to start this episode with the s...
2055,On the Shoulders of Giants,https://www.npr.org/2019/02/12/693878396/on-th...,When Colin Kaepernick stopped standing for the...,2345.0,693878396,"[<div aria-label=""Transcript"" class=""transcrip...",I would say that the purpose for us understan...


In [53]:
text = transcripts['cleaned_transcripts'][0][1051:2063]
text

"It'll be incumbent Republican Greg Abbott against Democrat Beto O'Rourke. Many other high-profile primary races are headed to runoffs, including Democratic Congressman Henry Cuellar, who is one of Congress's most conservative Democrats in the entire body. He'll be facing progressive challenger Jessica Cisneros. But it was also the first Election Day since sweeping election changes in the state that drastically change how a lot of people cast ballots. Before we get into the impact that that might have had, Ashley, I'm wondering if you can just start off for people who aren't, you know, in the weeds the same way you are into Texas election law. What changes were made since the 2020 election to how people vote there? Right. So lawmakers here passed SB 1. It - and that's a pretty massive overhaul of the state's election code, actually. So it changed a lot of things. For one, when it comes to in-person voting, the state set new limits for when polls can be open. It also got rid of drive-th

In [54]:
encoding = t5_tokenizer.encode("""summarize: """ + text, return_tensors='tf')


outputs = t5.generate(encoding,
                      num_beams=4, 
                      no_repeat_ngram_size=2,
                      min_length=30,
                      max_length=100,
                      early_stopping=True)

summarization = t5_tokenizer.decode(outputs[0])

In [55]:
outputs

<tf.Tensor: shape=(1, 62), dtype=int32, numpy=
array([[    0,    34,    47,    92,     8,   166,  4356,   239,   437,
            3, 23164,  4356,  1112,    16,     8,   538,     3,     5,
        23419,   270,  2804,     3,     9,  2876,    24, 24173,  2130,
          149,   151,  4061, 16524,     7,    16,     3, 10354,     9,
            7,     6,   379,    16,    18,  6075, 10601,     6,  1262,
           18, 11258, 10601,    11,     8,     3,     7,    35,   342,
           31,     7,  4514,    30,  2191,   190,     7,     5]],
      dtype=int32)>

In [56]:
summarization

"<pad> it was also the first election day since sweeping election changes in the state. lawmakers here passed a bill that drastically changed how people cast ballots in texas, including in-person voting, drive-through voting and the senate's ban on driving throughs."

## Paulowoicho T5 podcast summarization model

### Helpful links:
- https://metatext.io/models/paulowoicho-t5-podcast-summarisation
- https://huggingface.co/paulowoicho/t5-podcast-summarisation

### Fine Tuning links to look at later: 
- https://github.com/datasci-w266/2022-spring-main/blob/master/materials/walkthrough_notebooks/bert-finetuning/BERT_Fine-tuning.ipynb
- https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb


In [57]:
import transformers
from transformers import pipeline
from transformers import AutoModel, AutoTokenizer 
from transformers import TFAutoModelForSeq2SeqLM, TFT5ForConditionalGeneration

In [58]:
model_name = "paulowoicho/t5-podcast-summarisation" 


In [62]:
#!pip install torch
#!pip3 install torch torchvision
import torch

In [63]:
model = AutoModel.from_pretrained(model_name)
tokenizer= AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at paulowoicho/t5-podcast-summarisation were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [64]:
transcripts.head()

,title,link,description,duration,podcast_id,transcript,cleaned_transcripts
0,Changes To Texas Voting Rules Worry Older Vote...,https://www.npr.org/2022/03/02/1084043800/chan...,"Election officials said more than 15,000 mail-...",842.0,1084043800,"[<div aria-label=""Transcript"" class=""transcrip...","Hi. My name is Joey (ph), and I'm a bike mech..."
1,"At The State Of The Union, Biden Wooed Moderat...",https://www.npr.org/2022/03/02/1083868631/at-t...,"In his speech, the president emphasized polici...",1141.0,1083868631,"[<div aria-label=""Transcript"" class=""transcrip...",Hey there. It's the NPR POLITICS PODCAST. I'...
2,"""Awake Not Woke"": How Republicans Are Defining...",https://www.npr.org/2022/02/28/1083491401/-awa...,At last week's Conservative Political Action C...,828.0,1083491401,"[<div aria-label=""Transcript"" class=""transcrip...","(Speaking Catalan). And I'm her mother, Gina ..."
3,Ketanji Brown Jackson Is The First Black Woman...,https://www.npr.org/2022/02/25/1083180663/keta...,Jackson's experiences working as a public defe...,1571.0,1083180663,"[<div aria-label=""Transcript"" class=""transcrip...","Hi, this is Teresa (ph) on Jekyll Island, Ga...."
4,US Responds To Russian Invasion Of Ukraine Wit...,https://www.npr.org/2022/02/24/1082922707/us-r...,In a speech at the White House Thursday aftern...,796.0,1082922707,"[<div aria-label=""Transcript"" class=""transcrip...",Hey there. It's the NPR POLITICS PODCAST. It...


In [89]:
text = transcripts['cleaned_transcripts'][0][1051:2063]
text

"It'll be incumbent Republican Greg Abbott against Democrat Beto O'Rourke. Many other high-profile primary races are headed to runoffs, including Democratic Congressman Henry Cuellar, who is one of Congress's most conservative Democrats in the entire body. He'll be facing progressive challenger Jessica Cisneros. But it was also the first Election Day since sweeping election changes in the state that drastically change how a lot of people cast ballots. Before we get into the impact that that might have had, Ashley, I'm wondering if you can just start off for people who aren't, you know, in the weeds the same way you are into Texas election law. What changes were made since the 2020 election to how people vote there? Right. So lawmakers here passed SB 1. It - and that's a pretty massive overhaul of the state's election code, actually. So it changed a lot of things. For one, when it comes to in-person voting, the state set new limits for when polls can be open. It also got rid of drive-th

In [83]:
#inputs = tokenizer(text, return_tensors="pt")

In [84]:
#outputs = model(**inputs)

In [90]:
summarizer = pipeline("summarization", model="paulowoicho/t5-podcast-summarisation", tokenizer="paulowoicho/t5-podcast-summarisation")
summary = summarizer(text, min_length=5, max_length=20)

print(summary[0]['summary_text'])

this was the first election day that sweeping election changes in the state dramatically changed how people cast
